# RAG로 AI 와인 소믈리에 함수화 하기

## 환경변수 로딩

In [38]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [16]:
# OPENAI_EMBEDDING_MODEL, OPENAI_API_KEY

## 필요한 모듈 로딩

In [7]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

## 1. LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력 : 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체 사용하기

In [24]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert who analyzes food images.
        When a user provides an image of a dish,
        identify the commonly recognized name of the dish, and
        clearly and concisely describe its flavor, focusing on the cooking method, texture, aroma, and balance of taste.
        If there is any uncertainty, base your analysis on the most likely dish, avoid definitive claims, and maintain a professional, expert tone.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래의 이미지의 요리에 대한 요리명과 요리의 풍미를 설명해 주세요.
            출력형태 :
            요리명:
            요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.        
        ])
    ])
    
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GEMINI_API_KEY
    )
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser

    return chain

In [25]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://thumbnail.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/9d0d/fd3f0d77757f64b2eba0905dcdd85051932ec1ab5e6afc0c3246f403fabc.jpg"
}

# res = r1.invoke(input_data)

In [26]:
res = r1.invoke(input_data)

In [27]:
res

'요리명: 스테이크\n\n요리의 풍미:\n이 요리는 고온에서 완벽하게 시어링 되어 겉은 먹음직스러운 캐러멜 색으로 바삭하게 익고, 속은 육즙이 풍부한 미디엄 레어 또는 미디엄 굽기로 부드럽게 조리된 소고기 스테이크입니다. 구워진 표면의 고소하고 진한 육향과 함께, 부드럽고 촉촉한 속살은 입안에서 녹아내리는 듯한 뛰어난 식감을 선사합니다. 굵은 해염과 갓 부순 통후추는 고기의 깊은 감칠맛을 끌어올리며 적절한 짠맛과 은은한 알싸함을 더합니다. 신선한 로즈마리는 상쾌하고 향긋한 허브 아로마를 부여하여 묵직한 육류의 풍미에 섬세하고 균형 잡힌 맛을 더해줍니다.'

In [28]:
res = """
'요리명: 스테이크\n\n요리의 풍미:\n이 요리는 고온에서 완벽하게 시어링 되어 겉은 먹음직스러운 캐러멜 색으로 바삭하게 익고, 속은 육즙이 풍부한 미디엄 레어 또는 미디엄 굽기로 부드럽게 조리된 소고기 스테이크입니다. 구워진 표면의 고소하고 진한 육향과 함께, 부드럽고 촉촉한 속살은 입안에서 녹아내리는 듯한 뛰어난 식감을 선사합니다. 굵은 해염과 갓 부순 통후추는 고기의 깊은 감칠맛을 끌어올리며 적절한 짠맛과 은은한 알싸함을 더합니다. 신선한 로즈마리는 상쾌하고 향긋한 허브 아로마를 부여하여 묵직한 육류의 풍미에 섬세하고 균형 잡힌 맛을 더해줍니다.'
"""

## 2. 요리에 가장 잘 어울리는 wine top 5 검색
- pinecone 벡터 db 저장되어 있음
- index : wine-reviews, namespace : wine-reviews-ns

In [43]:
# 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 요리에 대한 풍미(설명) 들어오면,
# 벡터 db에 인덱할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 벡터 생성
embedding = OpenAIEmbeddings(
    model = OPENAI_EMBEDDING_MODEL,
    api_key = OPENAI_API_KEY
 )

# 벡터 db에서 유사도계산, top-5 검색
# 벡터 db 객체 생성
vector_db = PineconeVectorStore(
    embedding = embedding,
    index_name = PINECONE_INDEX_NAME,
    namespace = PINECONE_NAMESPACE,
    pinecone_api_key = PINECONE_API_KEY
)
# 벡터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 질문
print("질문 : ", query)
print("-"*50)
results = vector_db.similarity_search(query, k=5)  # top-5 검색
results

질문 :  
'요리명: 스테이크

요리의 풍미:
이 요리는 고온에서 완벽하게 시어링 되어 겉은 먹음직스러운 캐러멜 색으로 바삭하게 익고, 속은 육즙이 풍부한 미디엄 레어 또는 미디엄 굽기로 부드럽게 조리된 소고기 스테이크입니다. 구워진 표면의 고소하고 진한 육향과 함께, 부드럽고 촉촉한 속살은 입안에서 녹아내리는 듯한 뛰어난 식감을 선사합니다. 굵은 해염과 갓 부순 통후추는 고기의 깊은 감칠맛을 끌어올리며 적절한 짠맛과 은은한 알싸함을 더합니다. 신선한 로즈마리는 상쾌하고 향긋한 허브 아로마를 부여하여 묵직한 육류의 풍미에 섬세하고 균형 잡힌 맛을 더해줍니다.'

--------------------------------------------------


[Document(id='76616168-3be5-40db-83f8-65f6a1239819', metadata={'row': 40294.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=": 40294\ncountry: France\ndescription: This is a pretty, elegant Côte Rôtie, only medium in body, but with a lovely silky texture and a tart, lingering finish. Herbal aromas suggest green olives, and there's a hint of bacon smoke to go with flavors of tart raspberries and cherries. Drink now–2020.\ndesignation: \npoints: 92\nprice: 138.0\nprovince: Rhône Valley\nregion_1: Côte Rôtie\nregion_2: \ntaster_name: Joe Czerwinski\ntaster_twitter_handle: @JoeCz\ntitle: Michel & Stéphane Ogier 2010  Côte Rôtie\nvariety: Syrah\nwinery: Michel & Stéphane Ogier"),
 Document(id='d59719f5-5021-4220-a566-e51338c1ec7e', metadata={'row': 5664.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=": 5664\ncountry: US\ndescription: The winery's annual stainless-steel bottling, this is showy rather than reserved, with ripe Asian pear, tropical f

In [44]:
context = ""

context = "\n".join([doc.page_content for doc in results])

In [47]:
# print(context)

In [50]:
def search_wines(query):
    
    embedding = OpenAIEmbeddings(
         model = OPENAI_EMBEDDING_MODEL
     )
    
    # 벡터 db에서 유사도계산, top-5 검색
    # 벡터 db 객체 생성
    vector_db = PineconeVectorStore(
        embedding = embedding,
        index_name = PINECONE_INDEX_NAME ,
        namespace = PINECONE_NAMESPACE
    )
    # 벡터 db에서 질문과 가장 유사한, top-5 검색하기
    
    # print("질문 : ", query)
    # print("-"*50)
    
    results = vector_db.similarity_search(query, k=5)  # top-5 검색
    context = "\n".join([doc.page_content for doc in results])
    # print(results)

    # 결과 출력
    return {
        "query" : query,
        "wine_reviews" : context
    }

In [52]:
# 요리에 어울리는 와인 top-5 검색결과를 리턴하는 함수 호출

query = res  # 질문

topk_results = search_wines(query)

In [54]:
print(topk_results)

{'query': "\n'요리명: 스테이크\n\n요리의 풍미:\n이 요리는 고온에서 완벽하게 시어링 되어 겉은 먹음직스러운 캐러멜 색으로 바삭하게 익고, 속은 육즙이 풍부한 미디엄 레어 또는 미디엄 굽기로 부드럽게 조리된 소고기 스테이크입니다. 구워진 표면의 고소하고 진한 육향과 함께, 부드럽고 촉촉한 속살은 입안에서 녹아내리는 듯한 뛰어난 식감을 선사합니다. 굵은 해염과 갓 부순 통후추는 고기의 깊은 감칠맛을 끌어올리며 적절한 짠맛과 은은한 알싸함을 더합니다. 신선한 로즈마리는 상쾌하고 향긋한 허브 아로마를 부여하여 묵직한 육류의 풍미에 섬세하고 균형 잡힌 맛을 더해줍니다.'\n", 'wine_reviews': ": 40294\ncountry: France\ndescription: This is a pretty, elegant Côte Rôtie, only medium in body, but with a lovely silky texture and a tart, lingering finish. Herbal aromas suggest green olives, and there's a hint of bacon smoke to go with flavors of tart raspberries and cherries. Drink now–2020.\ndesignation: \npoints: 92\nprice: 138.0\nprovince: Rhône Valley\nregion_1: Côte Rôtie\nregion_2: \ntaster_name: Joe Czerwinski\ntaster_twitter_handle: @JoeCz\ntitle: Michel & Stéphane Ogier 2010  Côte Rôtie\nvariety: Syrah\nwinery: Michel & Stéphane Ogier\n: 5664\ncountry: US\ndescription: The winery's annual stainless-steel bottling, this is showy r

In [ ]:
# 함수 1 : 이미지 업로딩, LLM활용 설명 리턴
# 함수 2 : 요리 설명 입력 > top5 리턴
# 함수 3 : 요리 설명, top5 입력 > 와인 추천